In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
import numpy as np
import random
from tqdm import tqdm
from deepmatch.models import ComiRec, NCF, YoutubeDNN
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler
import tensorflow as tf
import gc

from deepctr.feature_column import DenseFeat

pad_sequences = tf.keras.utils.pad_sequences
SEQ_LEN = 10

2024-04-16 00:03:02.229423: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 00:03:02.371264: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-16 00:03:02.852235: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-04-16 00:03:02.852418: W tensorflow/compiler/xl

In [3]:
import load_data

In [4]:
data, movies = load_data.load_25m()

In [5]:
movies

,movieId,title,genres,hot,grade,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6,3,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,4,3,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,4,3,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3,2,1995
4,5,Father of the Bride Part II (1995),Comedy,4,3,1995
...,...,...,...,...,...,...
59042,209157,We (2018),Drama,1,1,2018
59043,209159,Window of the Soul (2001),Documentary,1,2,2001
59044,209163,Bad Poems (2018),Comedy|Drama,1,4,2018
59045,209169,A Girl Thing (2001),(no genres listed),1,2,2001


In [6]:
lbe = LabelEncoder()

In [7]:
movies["int_genres"] = lbe.fit_transform(movies["genres"]) + 1

In [8]:
data["genres"] = lbe.transform(data["genres"])+1

In [9]:
data["userId"] = data["user_id"]
data["movieId"] = data["movie_id"]

In [10]:
user_id_size =  data['userId'].max()+1
movie_id_size = data['movieId'].max()+1
genre_id_size = int(data['genres'].max())+1
grade_size = int(data['grade'].max()) + 1
hot_size = int(data['hot'].max()) + 1
year_size = int(data['year'].max()) + 1
embedding_dim = 64

In [11]:
user_feature_columns = [
                        # SparseFeat('user_id', user_id_size, 16),
                        VarLenSparseFeat(SparseFeat('hist_movie_id', movie_id_size, embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
                        VarLenSparseFeat(SparseFeat('hist_genres', genre_id_size, embedding_dim,
                                                   embedding_name="genres"), SEQ_LEN, 'mean', 'hist_len'),
                        VarLenSparseFeat(SparseFeat('hist_hot', genre_id_size, embedding_dim,
                                                   embedding_name="genres"), SEQ_LEN, 'mean', 'hist_len'),
                            VarLenSparseFeat(SparseFeat('hist_grade', genre_id_size, embedding_dim,
                                                   embedding_name="genres"), SEQ_LEN, 'mean', 'hist_len'),
                        VarLenSparseFeat(SparseFeat('hist_year', year_size, embedding_dim,
                                                   embedding_name="genres"), SEQ_LEN, 'mean', 'hist_len'),
                        ]
item_feature_columns = [SparseFeat('movie_id', movie_id_size, embedding_dim),]

In [12]:
def get_train_input_for_user(user_df):
    user_df = user_df.sort_values("timestamp")
    user_df = user_df[-50:] 
    user_id = user_df["user_id"].to_list()[0]
    tmp = [user_df[:x+1] for x in range(len(user_df))]
    input_data = []
    for record in tmp:
        sample = {
            "history_movie_id":np.array(record["movie_id"].tolist()[:-1]),
            "history_genre_id": np.array(record["genres"].tolist()[:-1]),
            "history_hot":np.array(record["hot"].tolist()[:-1]),
            "history_grade": np.array(record["grade"].tolist()[:-1]),
            "history_year": np.array(record["year"].tolist()[:-1]),
            "movie_id":np.array(record["movie_id"].tolist()[-1]),
            # np.array(record["genres"].tolist()[:-1]) # history genre list
        }
        input_data.append(sample)
    # train, test = input_data[:-1], input_data[-1:]
    result_df = pd.DataFrame(input_data)
    result_df["label"] = 1 # tmp for now
    result_df["hist_len"] = SEQ_LEN
    result_df[["label", "hist_len"]] = result_df[["label", "hist_len"]].astype("int8")
    return result_df

def get_test_input_for_user(user_df):
    #user_df = user_df.sort_values("timestamp")
    # user_df = user_df[-SEQ_LEN:] 
    user_id = user_df["user_id"].to_list()[0]
    
    input_data = []
    record = user_df
    sample = {
        "history_movie_id":np.array(record["movie_id"].tolist()[:-1]),
        "history_genre_id": np.array(record["genres"].tolist()[:-1]),
            "history_hot":np.array(record["hot"].tolist()[:-1]),
            "history_grade": np.array(record["grade"].tolist()[:-1]),
        "history_year": np.array(record["year"].tolist()[:-1]),
        "movie_id":np.array(record["movie_id"].tolist()[-1]),
        # np.array(record["genres"].tolist()[:-1]) # history genre list
    }
    input_data.append(sample)
    # train, test = input_data[:-1], input_data[-1:]
    result_df = pd.DataFrame(input_data)
    result_df["label"] = 1 # tmp for now
    result_df["hist_len"] = SEQ_LEN
    return result_df

In [13]:
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
SEQ_LEN=10

/root/.cache/pypoetry/virtualenvs/cinepaw-P6zxCTLW-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
data["user_id"] = data["userId"]
data["movie_id"] = data["movieId"]

In [15]:
df = data.groupby("user_id").progress_apply(lambda x:get_test_input_for_user(x))

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 162541/162541 [01:00<00:00, 2678.44it/s]


In [16]:
train_set = data.groupby("user_id").progress_apply(lambda x:get_train_input_for_user(x))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 162541/162541 [12:18<00:00, 220.22it/s]


In [17]:
def construct_input(df):
    df["history_movie_id"] = pad_sequences(df["history_movie_id"], maxlen=SEQ_LEN, padding='pre', truncating='pre', value=0).tolist()
    df["history_genre_id"] = pad_sequences(df["history_genre_id"], maxlen=SEQ_LEN, padding='pre', truncating='pre', value=0).tolist()
    df["history_hot"] = pad_sequences(df["history_hot"], maxlen=SEQ_LEN, padding='pre', truncating='pre', value=0).tolist()
    df["history_grade"] = pad_sequences(df["history_grade"], maxlen=SEQ_LEN, padding='pre', truncating='pre', value=0).tolist()
    df["history_year"] = pad_sequences(df["history_year"], maxlen=SEQ_LEN, padding='pre', truncating='pre', value=0).tolist()
    
    
    df = df.sample(frac=1)
    
    model_input = {
        "hist_movie_id":np.array(df["history_movie_id"].tolist()),
        "hist_genres":np.array(df["history_genre_id"].tolist()),
        "hist_hot":np.array(df["history_hot"].tolist()),
        "hist_grade":np.array(df["history_grade"].tolist()),
        "hist_year":np.array(df["history_year"].tolist()),
        "movie_id": np.array(df["movie_id"].tolist()),
        "hist_len": np.array(df["hist_len"].tolist()),
    }
    label =  df["label"].tolist()
    return model_input, label

In [18]:
train_model_input, train_label = construct_input(train_set)

## COMI REC

In [19]:
import tensorflow as tf
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

from collections import Counter
train_counter = Counter(data['movie_id'])
item_count = [train_counter.get(i,0) for i in range(item_feature_columns[0].vocabulary_size)] #todo change this later
sampler_config = NegativeSampler('frequency',num_sampled=255,item_name="movie_id",item_count=item_count)
#model = YoutubeDNN(user_feature_columns, item_feature_columns, user_dnn_hidden_units=(128,64, embedding_dim), sampler_config=sampler_config)
model = ComiRec(user_feature_columns,
                item_feature_columns,
                k_max=2, 
                user_dnn_hidden_units=(128,64, embedding_dim),
                sampler_config=sampler_config)


model.compile(optimizer="adam", loss=sampledsoftmaxloss)
history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.01, use_multiprocessing=True)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2024-04-16 00:18:56.745412: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-04-16 00:18:56.745429: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: pop-os
2024-04-16 00:18:56.745431: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: pop-os
2024-04-16 00:18:56.745509: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.89.2
2024-04-16 00:18:56.745524: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.89.2
2024-04-16 00:18:56.745525: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 525.89.2


Train on 6947119 samples, validate on 70173 samples
Epoch 1/20


2024-04-16 00:19:00.326924: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 00:19:00.354575: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2024-04-16 00:19:00.439014: W tensorflow/c/c_api.cc:291] Operation '{name:'user_dnn/bias2/Assign' id:350 op device:{requested: '', assigned: ''} def:{{{node user_dnn/bias2/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](user_dnn/bias2, user_dnn/bias2/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them o

1817088/6947119 [======>.......................] - ETA: 34:10 - loss: 5.1074

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



6947119/6947119 [==============================] - 2754s 396us/sample - loss: 3.8285 - val_loss: 3.8672
Epoch 13/20
6947119/6947119 [==============================] - 2751s 396us/sample - loss: 3.8236 - val_loss: 3.8685
Epoch 14/20
6947119/6947119 [==============================] - 2754s 396us/sample - loss: 3.8199 - val_loss: 3.8539
Epoch 15/20
6947119/6947119 [==============================] - 2755s 397us/sample - loss: 3.8165 - val_loss: 3.8588
Epoch 16/20
5872128/6947119 [========================>.....] - ETA: 7:05 - loss: 3.8109

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



6947119/6947119 [==============================] - 2761s 397us/sample - loss: 3.8075 - val_loss: 3.8480
Epoch 19/20
6947119/6947119 [==============================] - 2759s 397us/sample - loss: 3.8027 - val_loss: 3.8399


In [20]:
history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.03, use_multiprocessing=True)

Train on 6806773 samples, validate on 210519 samples
Epoch 1/20
1071104/6806773 [===>..........................] - ETA: 38:00 - loss: 3.7792

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe7222a2b60>>
Traceback (most recent call last):
  File "/root/.cache/pypoetry/virtualenvs/cinepaw-P6zxCTLW-py3.10/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

## YoutubeDNN

In [132]:
import math

In [135]:
int(math.sqrt())

0

In [ ]:
int(sq

In [308]:
import tensorflow as tf
from deepmatch.models import ComiRec, NCF, YoutubeDNN
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

from collections import Counter
train_counter = Counter(data['movie_id'])
item_count = [int(math.sqrt(train_counter.get(i,0))) for i in range(item_feature_columns[0].vocabulary_size)] #todo change this later
sampler_config = NegativeSampler('frequency',num_sampled=255,item_name="movie_id",item_count=item_count)
#model = YoutubeDNN(user_feature_columns, item_feature_columns, user_dnn_hidden_units=(128,64, embedding_dim), sampler_config=sampler_config)
model = YoutubeDNN(user_feature_columns,
                item_feature_columns,
                user_dnn_hidden_units=(128,64, embedding_dim),
                sampler_config=sampler_config)


model.compile(optimizer="adam", loss=sampledsoftmaxloss)
history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.01, use_multiprocessing=True)

Train on 990206 samples, validate on 10003 samples
Epoch 1/20


2024-04-13 11:38:19.545958: W tensorflow/c/c_api.cc:291] Operation '{name:'training_8/Adam/dnn_2/bias1/v/Assign' id:8206 op device:{requested: '', assigned: ''} def:{{{node training_8/Adam/dnn_2/bias1/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training_8/Adam/dnn_2/bias1/v, training_8/Adam/dnn_2/bias1/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


989696/990206 [============================>.] - ETA: 0s - loss: 3.3094

/root/.cache/pypoetry/virtualenvs/cinepaw-P6zxCTLW-py3.10/lib/python3.10/site-packages/tensorflow/python/keras/engine/training.py:2455: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2024-04-13 11:38:33.231085: W tensorflow/c/c_api.cc:291] Operation '{name:'loss_5/AddN_1' id:7461 op device:{requested: '', assigned: ''} def:{{{node loss_5/AddN_1}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss_5/mul, loss_5/AddN)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


990206/990206 [==============================] - 14s 14us/sample - loss: 3.3087 - val_loss: 1.9468
Epoch 2/20
990206/990206 [==============================] - 13s 13us/sample - loss: 1.7730 - val_loss: 1.6539
Epoch 3/20
990206/990206 [==============================] - 13s 13us/sample - loss: 1.5864 - val_loss: 1.5479
Epoch 4/20
990206/990206 [==============================] - 13s 13us/sample - loss: 1.4908 - val_loss: 1.4859
Epoch 5/20
990206/990206 [==============================] - 13s 13us/sample - loss: 1.4218 - val_loss: 1.4166
Epoch 6/20
990206/990206 [==============================] - 13s 13us/sample - loss: 1.3684 - val_loss: 1.3854
Epoch 7/20
990206/990206 [==============================] - 13s 13us/sample - loss: 1.3301 - val_loss: 1.3527
Epoch 8/20
990206/990206 [==============================] - 13s 13us/sample - loss: 1.2959 - val_loss: 1.3477
Epoch 9/20
990206/990206 [==============================] - 13s 13us/sample - loss: 1.2680 - val_loss: 1.2992
Epoch 10/20
990206/99

## Single Test

In [21]:
from scipy.spatial.distance import cdist

In [22]:
all_movie_ids = movies["movieId"]
metadatas = movies.to_dict(orient="records")

all_item_model_input = {"movie_id": all_movie_ids}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

#user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

/root/.cache/pypoetry/virtualenvs/cinepaw-P6zxCTLW-py3.10/lib/python3.10/site-packages/tensorflow/python/keras/engine/training.py:2455: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2024-04-16 15:46:06.694992: W tensorflow/c/c_api.cc:291] Operation '{name:'lambda_3/Squeeze' id:613 op device:{requested: '', assigned: ''} def:{{{node lambda_3/Squeeze}} = Squeeze[T=DT_FLOAT, _has_manual_control_dependencies=true, squeeze_dims=[1]](lambda_3/GatherV2)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [23]:
movies[(movies["genres"]=="Horror")&(movies["year"]>1970)]\
    .sort_values(["hot"]).tail(10)

,movieId,title,genres,hot,grade,year,int_genres
2426,2517,Christine (1983),Horror,3,3,1983,1554
2422,2513,Pet Sematary (1989),Horror,3,3,1989,1554
2368,2459,"Texas Chainsaw Massacre, The (1974)",Horror,3,3,1974,1554
2029,2119,Maximum Overdrive (1986),Horror,3,2,1986,1554
1910,1999,"Exorcist III, The (1990)",Horror,3,2,1990,1554
1896,1985,Halloween 4: The Return of Michael Myers (1988),Horror,3,2,1988,1554
1895,1984,Halloween III: Season of the Witch (1982),Horror,3,2,1982,1554
3814,3917,Hellraiser (1987),Horror,3,3,1987,1554
175,177,Lord of Illusions (1995),Horror,3,2,1995,1554
1225,1258,"Shining, The (1980)",Horror,4,4,1980,1554


In [36]:
watched_titles = ["Blair Witch Project, The (1999)", "Exorcist, The (1973)", "Friday the 13th Part V: A New Beginning (1985)"]

In [100]:
watched_titles = movies[(movies["genres"]=="Horror")&(movies["year"]>1990)]\
    .sort_values(["hot"]).tail(10)["title"].tolist()

In [101]:
watched_titles = watched_titles[0:10]

In [102]:
user_df = movies[movies["title"].isin(watched_titles)]
user_df["genres"] = lbe.transform(user_df["genres"]) + 1
model_input = {
    "hist_movie_id": pad_sequences(np.array([user_df["movieId"].tolist()]), maxlen=SEQ_LEN, padding='pre', truncating='pre', value=0),
    "hist_genres": pad_sequences(np.array([user_df["genres"].tolist()]), maxlen=SEQ_LEN, padding='pre', truncating='pre', value=0),
    "hist_hot": pad_sequences(np.array([user_df["hot"].tolist()]), maxlen=SEQ_LEN, padding='pre', truncating='pre', value=0),
    "hist_grade": pad_sequences(np.array([user_df["grade"].tolist()]), maxlen=SEQ_LEN, padding='pre', truncating='pre', value=0),
    "hist_year": pad_sequences(np.array([user_df["year"].tolist()]), maxlen=SEQ_LEN, padding='pre', truncating='pre', value=0),
    "hist_len": np.array([len(user_df["genres"].tolist())])
}

/tmp/ipykernel_2131206/3737173683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_df["genres"] = lbe.transform(user_df["genres"]) + 1


In [103]:
user_embs = user_embedding_model.predict(model_input, batch_size=2 ** 12)

In [104]:
watched_titles

['The Witch (2015)',
 'Ghost Ship (2002)',
 'House of 1000 Corpses (2003)',
 'Texas Chainsaw Massacre, The (2003)',
 'Crimson Peak (2015)',
 'Ju-on: The Grudge (2002)',
 'Hostel (2005)',
 'The Conjuring 2 (2016)',
 'It (2017)',
 'Lord of Illusions (1995)']

In [106]:
matrix = item_embs
distance = cdist(np.array([user_embs[0][0]]), matrix).min(axis=0)
movies["distance"] = distance
movies.sort_values("distance").head(20)

,movieId,title,genres,hot,grade,year,int_genres,distance
16210,85788,Insidious (2010),Fantasy|Horror|Thriller,3,3,2010,1522,7.515076
18568,97188,Sinister (2012),Horror|Thriller,3,3,2012,1575,7.534821
26508,128838,Crimson Peak (2015),Horror,3,3,2015,1554,7.541782
38646,160571,Lights Out (2016),Horror,2,3,2016,1554,7.554824
6584,6707,Cabin Fever (2002),Horror|Thriller,3,2,2002,1575,7.556892
6757,6880,"Texas Chainsaw Massacre, The (2003)",Horror,3,2,2003,1554,7.562357
13551,70159,Orphan (2009),Drama|Horror|Mystery|Thriller,3,3,2009,1457,7.584973
6178,6290,House of 1000 Corpses (2003),Horror,3,2,2003,1554,7.586911
6572,6695,Jeepers Creepers 2 (2003),Horror|Thriller,2,2,2003,1575,7.598481
21960,113565,"Sacrament, The (2013)",Horror|Thriller,2,3,2013,1575,7.610946


In [432]:
matrix = item_embs
distance = cdist(np.array([user_embs[0][1]]), matrix).min(axis=0)
movies["distance"] = distance
movies.sort_values("distance").head(20)

,movieId,title,genres,hot,grade,year,int_genres,distance
607,624,Condition Red (1995),Action|Drama|Thriller,1,3,1995,72,10.890239
3384,3624,Shanghai Noon (2000),Action,2,3,2000,1,10.890385
3097,3323,Chain of Fools (2000),Comedy|Crime,1,2,2000,178,10.898260
3383,3623,Mission: Impossible 2 (2000),Action|Thriller,3,3,2000,93,10.906598
2556,2761,"Iron Giant, The (1999)",Animation|Children's,2,4,1999,145,10.913048
3409,3651,"Blood Spattered Bride, The (La Novia Ensangren...",Horror,1,0,1972,275,10.917136
259,266,Legends of the Fall (1994),Drama|Romance|War|Western,2,3,1994,257,10.925069
3251,3484,"Skulls, The (2000)",Thriller,2,2,2000,299,10.929373
396,410,Addams Family Values (1993),Comedy,2,2,1993,177,10.946220
3463,3705,Bird on a Wire (1990),Action|Adventure|Romance|Thriller,2,2,1990,32,10.958496


In [431]:
watched_titles

["Nightmare on Elm Street Part 2: Freddy's Revenge, A (1985)",
 'Nightmare on Elm Street 3: Dream Warriors, A (1987)',
 'Pet Sematary II (1992)',
 'Pet Sematary (1989)',
 'Nightmare on Elm Street 4: The Dream Master, A (1988)',
 'Nightmare on Elm Street 5: The Dream Child, A (1989)',
 "Freddy's Dead: The Final Nightmare (1991)",
 'Christine (1983)',
 'Blair Witch Project, The (1999)',
 'Shining, The (1980)']

In [458]:
ComiRec

<function deepmatch.models.comirec.ComiRec(user_feature_columns, item_feature_columns, k_max=2, p=100, interest_extractor='sa', add_pos=True, user_dnn_hidden_units=(64, 32), dnn_activation='relu', dnn_use_bn=False, l2_reg_dnn=0, l2_reg_embedding=1e-06, dnn_dropout=0, output_activation='linear', sampler_config=None, seed=1024)>